AMUSE tutorial on high-order bridge
====================

Hierarchical coupling strategies are fundamental parts of AMUSE.
It enables us to combined the output of a wide variety of solvers into a homogeneous solution.
In this example we will be nesting multiple bridges, to show the power of bridge.

In [ ]:
import numpy
from amuse.units import (units, constants)
#from amuse.lab import Particles

In [ ]:
def orbital_period(Mtot, a):
    return (((4 * numpy.pi**2) * a**3)/(constants.G * Mtot)).sqrt()

from amuse.ext.orbital_elements import new_binary_from_orbital_elements
from amuse.ext.orbital_elements import orbital_elements_from_binary

def get_star_planet_and_moon():
    Mstar = 1.0 | units.MSun
    Mplanet = 1.0 | units.MJupiter
    a_planet = 5.2 | units.au
    e_planet = 0.0
    bodies = new_binary_from_orbital_elements(Mstar, Mplanet, a_planet, e_planet,
                                             G=constants.G)
    bodies[0].name = "star"
    bodies[1].name = "planet"
    planet = bodies[bodies.name=="planet"]
    RH_planet = a_planet * (1.0 - e_planet) * (Mplanet / (3 * Mstar))**(1./3.)
    a_moon = 0.1 * RH_planet
    e_moon = 0.0
    Mmoon = 0.01 * Mplanet
    Pmoon = orbital_period(Mplanet+Mmoon, a_moon)
    moon = new_binary_from_orbital_elements(planet.mass, 
                                            Mmoon, a_moon, e_moon,
                                            G=constants.G)
    moon.position -= moon[0].position
    moon.velocity -= moon[0].velocity
    moon = moon[1].as_set()
    moon.position += planet.position
    moon.velocity += planet.velocity
    moon.semimajor_axis = a_moon
    moon.name = "moon"
    bodies.add_particle(moon)
    bodies.move_to_center()
    return bodies
bodies = get_star_planet_and_moon()
print(bodies)

Now we have the orbits of the three particles, star, planet and moon. We now want to make a disk around the moon.

In [ ]:
from amuse.ext.protodisk import ProtoPlanetaryDisk

from amuse.lab import nbody_system


def make_disk_around_celestial_body(moon, Mplanet):

    R = 1 | units.au
    a_moon = moon.semimajor_axis
    e_moon = 0.0
    Mmoon = moon.mass.sum()
    RH_moon = a_moon * (1.0 - e_moon) * (Mmoon / (3 * Mplanet))**(1./3.)
    converter = nbody_system.nbody_to_si(Mmoon, R)
    Ndisk = 1000
    Rin = 0.03 * RH_moon
    Rout = 0.3 * RH_moon
    Pinner = orbital_period(Mmoon, Rin)
    Mdisk = 0.01 * Mmoon

    disk = ProtoPlanetaryDisk(Ndisk,
                              convert_nbody=converter,
                              Rmin=Rin/R,
                              Rmax=Rout/R,
                              q_out=10.0,
                              discfraction=Mdisk/Mmoon).result
    disk.name = "disk "
    disk.move_to_center()
    disk.position += moon.position
    disk.velocity += moon.velocity

    masses = Mdisk / float(Ndisk)
    disk.mass = masses
    rho = 3.0 | (units.g / units.cm**3)
    disk.radius = (disk.mass / (4 * rho))**(1./3.)
    return disk, Pinner, converter

planet = bodies[bodies.name=="planet"]
moon = bodies[bodies.name=="moon"]
disk, Pinner, converter = make_disk_around_celestial_body(moon, 
                                               planet.mass.sum())

In [ ]:
from amuse.plot import scatter
def plot(moon, disk):
    scatter(disk.x, disk.y)
    scatter(moon.x, moon.y, c='r')
plot(moon, disk)

In [ ]:
from amuse.community.huayno.interface import Huayno
from amuse.community.fi.interface import Fi
gravity = Huayno(converter)
gravity.particles.add_particles(bodies-moon)
channel = {"from stars": bodies.new_channel_to(gravity.particles),
            "to_stars": gravity.particles.new_channel_to(bodies)}

hydro = Fi(converter, mode="openmp")
hydro.parameters.use_hydro_flag = True
hydro.parameters.radiation_flag = False
hydro.parameters.gamma = 1
hydro.parameters.isothermal_flag = True
hydro.parameters.integrate_entropy_flag = False
hydro.parameters.timestep = 0.01*Pinner 
hydro.parameters.verbosity = 0
hydro.parameters.eps_is_h_flag = False  # h_smooth is constant
eps = 0.1 | units.au
hydro.parameters.gas_epsilon = eps
hydro.parameters.sph_h_const = eps

hydro.particles.add_particles(disk)
hydro.dm_particles.add_particles(moon.as_set())
channel.update({"from_disk": disk.new_channel_to(hydro.particles)})
channel.update({"to_disk": hydro.particles.new_channel_to(disk)})
channel.update({"from_moon": moon.new_channel_to(hydro.dm_particles)})
channel.update({"to_moon": hydro.dm_particles.new_channel_to(moon)})

bodies.add_particles(disk)

In [ ]:
from amuse.couple import bridge
from amuse.ext.composition_methods import *
gravhydro = bridge.Bridge(use_threading=False)  # , method=SPLIT_4TH_S_M4)
gravhydro.add_system(gravity, (hydro,))
gravhydro.add_system(hydro, (gravity,))
gravhydro.timestep = 0.2 * Pinner

In [ ]:
from amuse.ext.composition_methods import *
from amuse.ext.orbital_elements import orbital_elements_from_binary

def gravity_hydro_bridge(gravity, hydro, gravhydro, bodies,
                         t_end):

    gravity_initial_total_energy = gravity.get_total_energy() + hydro.get_total_energy()
    model_time = 0 | units.Myr
    dt = 0.012 | units.yr  # 1.0*Pinner
    while model_time < t_end:

        model_time += dt

        orbit_planet = orbital_elements_from_binary(bodies[:2], G=constants.G)
        orbit_moon = orbital_elements_from_binary(bodies[1:3], G=constants.G)
        print("Planet:", "ae=", orbit_planet[2].in_(units.AU), orbit_planet[3])
        print("Moon:", "ae=", orbit_moon[2].in_(units.AU), orbit_moon[3])
        
        dE_gravity = gravity_initial_total_energy / (
            gravity.get_total_energy() + hydro.get_total_energy()
        )
        print("Time:", model_time.in_(units.day), \
              "dE=", dE_gravity)  # , dE_hydro

        gravhydro.evolve_model(model_time)
        channel["to_stars"].copy()
        channel["to_disk"].copy()
        channel["to_moon"].copy()
        
        print("S=", bodies[:3])
        print("g=", gravity.particles)
        print(gravity.particles.y.in_(units.au), moon.y.in_(units.au))

    gravity.stop()
    hydro.stop()

t_end = 1.0 | units.yr
gravity_hydro_bridge(gravity, hydro, gravhydro, 
                     bodies, t_end)

In [ ]:
plot(moon, disk)

You have adopted the star-planet-moon system with disk. Instead of hyrarchical_bridge.py where the disk was integrated using a direct N-body code, the disk is now a gaseous.


Assignments and questions:
---------------

### Assignment 1:


### Question 1:
